In [99]:
import sys
import os

# Chemin vers le dossier racine du projet
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(BASE_DIR)

import app_database as db
print(db.__file__)

import importlib                                            # To reload the modules
importlib.reload(db)                                        # Reload the db module

import sqlite3
import os
import datetime

import regex as re

import pandas as pd
print(pd.__version__)

def read_excel_file(file_path):
    # Load the Excel file
    df = pd.read_excel(file_path, header=None)
    
    #display(df)
    
    first_row = df.iloc[0]              # Identify columns that have non-null values in the first row
    columns_with_data = first_row.notnull()
    
    df = df.loc[:, columns_with_data]   # Keep only columns with non-null values in the first row
    #display(df)
    
    df.columns = df.iloc[0]             # Set the first row as column headers
    df = df.iloc[1:]                    # Remove the first row which is now the header
    
    df = df.dropna(how='all')           # Further filter rows based on non-null values in all columns
    
    display(df)

    return df

c:\Users\223133654\Documents\Application\power-meter-selector\app_database.py
2.2.3


In [100]:
file_path = "C:/Users/223133654/Documents/Application/power-meter-selector/data/raw/GE Grid Automation EPM Meters Order Codes - 03-11-2025.xlsx"
df = read_excel_file(file_path)

,Reference,Enclosure Option,EPM2200 Option,EPM2200 Communications,Description,Dependency Expression,Rule Type
1,PL2200,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,ENC120,NaN,NaN,"NEMA1 Rated - Indoor, Single Meter Enclosure, ...",NaN,NaN
3,NaN,ENC277,NaN,NaN,"NEMA1 Rated - Indoor, Single Meter Enclosure, ...",NaN,NaN
4,NaN,ENN120,NaN,NaN,"NEMA4X Rated - Indoor/Outdoor, Single Meter En...",NaN,NaN
5,NaN,ENN277,NaN,NaN,"NEMA4X Rated - Indoor/Outdoor, Single Meter En...",NaN,NaN
6,NaN,XXXXXX,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,A1,NaN,Volts and Amps meter,NaN,NaN
8,NaN,NaN,B1,NaN,"Volts, Amps, Power and Frequency meter",NaN,NaN
9,NaN,NaN,C1,NaN,"Volts, Amps, Power, Frequency and Energy Count...",NaN,NaN
10,NaN,NaN,BN,NaN,"BACnet Volts, Amps, Power, Frequency and Energ...",NaN,NaN


In [ ]:
# Charger toutes les feuilles du fichier Excel
all_sheets = pd.read_excel(file_path, sheet_name=None, header=None)

for sheet_name, df in all_sheets.items():
    print(f"\n▶ Sheet_name : {sheet_name}")

    # Dataframe cleaning
    df = df.dropna(how='all')
    first_row = df.iloc[0]
    columns_with_data = first_row.notnull()
    df = df.loc[:, columns_with_data]
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    df = df.dropna(how='all')

    if df.empty:
        print(f"⚠ Empty sheet {sheet_name}. Skipping.")
        continue

    # Check for duplicate column names and remove them
    if df.columns.duplicated().any():
        print(f"⚠ Sheet {sheet_name} contains duplicate column names: {df.columns[df.columns.duplicated()].tolist()}")
        df = df.loc[:, ~df.columns.duplicated()]
        # TODO : check double column
        print(f"Duplicate columns removed, remaining columns: {df.columns.tolist()}")

    # --- Insert into `products` ---
    reference = df.iloc[0, 0]                       # <== First row, first column is the reference
    if pd.isna(reference) or str(reference).strip() == "":
        print(f"⚠ Empty reference in sheet {sheet_name}. Skipping.")
        continue

    name = sheet_name                               # <== Sheet name is the product name
    description = "No Description"                  # TODO <== Description of the product -> to be implemented
                                                    #! TODO ! It exists ! -> Description of the first line of the sheet
    if pd.isna(description) or str(description).strip() == "":
        description = "No Description"

    print(f"Inserting product : {reference}, {name}, {description}")

    try:
        db.add_element_to_table("products", [reference, name, description], True, conflict_mode='ignore')
    except sqlite3.IntegrityError:
        print(f"⚠ Product reference '{reference}' already exists, skipped.")
        continue

    product_id = db.get_last_id_from_table("products")

    # --- Insert into `product_options` and `options_rules` ---
    print("Columns available in the DataFrame:", df.columns)
    for column in df.columns[1:]:
        if str(column).strip().lower() in ["description", "dependency expression", "rule type"]:
            print(f"Skipping column '{column}'")
            continue

        category = column
        for index, option_code in df[column].dropna().items():
            option_code = str(option_code).strip()
            if option_code == "" or re.fullmatch(r'\|+\s*', option_code):
                continue  # Ignore empty/placeholder values like "|" or "||"

            option_description = None
            if 'Description' in df.columns:
                option_description = df.loc[index, 'Description']
                if pd.isna(option_description) or str(option_description).strip() == "":
                    option_description = None

            print(f"Inserting Product ID: {product_id}, Category: {category}, Option_code: {option_code}, Description: {option_description}")
            
            try:
                db.add_element_to_table("product_options", [product_id, category, option_code, option_description], True, conflict_mode='ignore')
            except sqlite3.IntegrityError:
                print(f"⚠ Option code '{option_code}' already exists, skipped.")
                continue

            # --- Insert rules if applicable ---
            dependency_expression = None
            rule_type = None
            if 'Dependency Expression' in df.columns:
                dependency_expression = df.loc[index, 'Dependency Expression']
                if pd.isna(dependency_expression) or str(dependency_expression).strip() == "":
                    dependency_expression = None

            if 'Rule Type' in df.columns:
                rule_type = df.loc[index, 'Rule Type']
                if pd.isna(rule_type) or str(rule_type).strip() == "":
                    rule_type = None

            if dependency_expression and rule_type:
                print(f"Inserting rule for Product ID: {product_id}, Option_code: {option_code}, Dependency: {dependency_expression}, Rule Type: {rule_type}")
                try:
                    db.add_element_to_table("options_rules", [product_id, option_code, dependency_expression, rule_type], True, conflict_mode='ignore')
                except sqlite3.IntegrityError:
                    print(f"⚠ Rule for option code '{option_code}' already exists, skipped.")
                    continue




▶ Sheet_name : EPM2200
Inserting product : PL2200, EPM2200, No Description
Columns available in the DataFrame: Index(['Reference', 'Enclosure Option', 'EPM2200 Option',
       'EPM2200 Communications', 'Description', 'Dependency Expression',
       'Rule Type'],
      dtype='object', name=0)
Inserting Product ID: 4, Category: Enclosure Option, Option_code: ENC120, Description: NEMA1 Rated - Indoor, Single Meter Enclosure, 120V
Inserting Product ID: 4, Category: Enclosure Option, Option_code: ENC277, Description: NEMA1 Rated - Indoor, Single Meter Enclosure, 277V
Inserting Product ID: 4, Category: Enclosure Option, Option_code: ENN120, Description: NEMA4X Rated - Indoor/Outdoor, Single Meter Enclosure, 120V
Inserting Product ID: 4, Category: Enclosure Option, Option_code: ENN277, Description: NEMA4X Rated - Indoor/Outdoor, Single Meter Enclosure, 277V
Inserting Product ID: 4, Category: Enclosure Option, Option_code: XXXXXX, Description: None
Inserting Product ID: 4, Category: EPM2200 O

In [103]:
db.print_table_by_name('products')
db.print_table_by_name('product_options')
db.print_table_by_name('options_rules')


----- Table: products -----


,id,reference,name,description
0,1,10000,Banana,Fruit
1,2,22222,Orange,Fruit
2,3,55555,Cucumber,Vegetable
3,4,PL2200,EPM2200,No_Description
4,5,PL4600,EPM4600,No_Description
5,6,PL6000,EPM6000,No_Description
6,7,PL6010,EPM6010,No_Description
7,8,PL7000,EPM7000,No_Description
8,9,PL7000P,EPM_7000P,No_Description
9,10,PL7000PT,EPM_7000PT,No_Description



----- Table: product_options -----


,id,product_id,category,option_code,description
0,1,10000,color,yellow,Yellow_banana
1,2,11111,color,red,Red_apple
2,3,11111,color,green,Green_apple
3,4,4,Enclosure_Option,ENC120,NEMA1_Rated___Indoor__Single_Meter_Enclosure__120V
4,5,4,Enclosure_Option,ENC277,NEMA1_Rated___Indoor__Single_Meter_Enclosure__277V
5,6,4,Enclosure_Option,ENN120,NEMA4X_Rated___Indoor_Outdoor__Single_Meter_Enclosure__120V
6,7,4,Enclosure_Option,ENN277,NEMA4X_Rated___Indoor_Outdoor__Single_Meter_Enclosure__277V
7,8,4,Enclosure_Option,XXXXXX,None
8,9,4,EPM2200_Option,A1,Volts_and_Amps_meter_
9,10,4,EPM2200_Option,B1,Volts__Amps__Power_and_Frequency_meter



----- Table: options_rules -----


,id,product_id,option_category,option_code,expression,rule_type
0,1,1,color,yellow,A AND (B OR C),required
1,2,2,color,red,X AND (Y OR Z),forbidden
2,3,3,color,green,L AND (M OR N),forbidden
3,4,4,EPM2200_Communications,S,A1_OR_B1_OR_C1,required
4,5,4,EPM2200_Communications,X,A1_OR_B1_OR_C1,required
5,6,4,EPM2200_Communications,B,BN,required
